In [ ]:
!pip install num2words

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import nltk
from glob import glob
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import num2words
from tqdm import tqdm
import pickle

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('corpus')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading corpus: Package 'corpus' not found in index
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
#Mounting drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
file1 = pd.read_csv('amisha.csv', on_bad_lines='skip', encoding='latin-1', engine='python')
file2 = pd.read_csv("twitter_data_aryan.csv", on_bad_lines='skip', encoding='latin-1', engine='python')
file3 = pd.read_csv("twitter_data_Harman.csv", on_bad_lines='skip', encoding='latin-1', engine='python') 
file4 = pd.read_csv("twitter_data_yash_bhargava.csv", on_bad_lines='skip', encoding='latin-1', engine='python') 
file5 = pd.read_csv("twitter_data_yash_tanwar.csv", on_bad_lines='skip', encoding='latin-1', engine='python') 
file6 = pd.read_csv("twitter_data_mayank.csv", on_bad_lines='skip', encoding='latin-1', engine='python')

In [ ]:
frames = [file1, file2, file3, file4, file5, file6]
final_pd = pd.concat(frames)

In [ ]:
print('Size = ', final_pd.shape)

Size =  (274803, 22)


In [ ]:
# dropping ALL duplicate values
final_pd.drop_duplicates(subset ="tweet", keep = False, inplace = True)
final_pd.fillna('', inplace=True)

In [ ]:
print('Size = ', final_pd.shape)

Size =  (224213, 22)


In [ ]:
#Converting to lowercase
def to_lower_case(text):
  text = text.lower()
  return text

#Substituting words with apostrophe with their possible base form
def substitute_text(text):
  text_list = []
  substitutes = {"aren't": 'are not', "can't": 'cannot', "couldn't": 'could not', "didn't": 'did not', "doesn't": 'does not', "don't": 'do not', "hadn't": 'had not', "hasn't": 'has not', "haven't": 'have not', "he'd": 'he would', "he'll": 'he will', "he's": 'he is', "i'd": 'i would', "i'll": 'i will', "i'm": 'i am', "isn't": 'is not', "it's": 'it is', "it'll": 'it will', "i've": 'i have', "let's": 'let us', "mightn't": 'might not', "mustn't": 'must not',"n't": 'not', "shan't": 'shall not', "she'd": 'she would', "she'll": 'she will', "she's": 'she is', "shouldn't": 'should not', "that's": 'that is', "there's": 'there is', "they'd": 'they would', "they'll": 'they will', "they're": 'they are', "they've": 'they have', "we'd": 'we would', "we're": 'we are', "weren't": 'were not', "we've": 'we have', "what'll": 'what will', "what're": 'what are', "what's": 'what is', "what've": 'what have', "where's": 'where is', "who'd": 'who would', "who'll": 'who will', "who're": 'who are', "who's": 'who is', "who've": 'who have', "won't": 'will not', "wouldn't": 'would not', "you'd": 'you would', "you'll": 'you will', "you're": 'you are', "you've": 'you have', "'re": ' are', "wasn't": 'was not', "we'll": 'we will', "'cause": 'because', "could've": 'could have', "how'd": 'how did', "how'd'y": 'how do you', "how'll": 'how will', "how's": 'how is', "I'd": 'I would', "I'd've": 'I would have', "I'll": 'I will', "I'll've": 'I will have', "I'm": 'I am', "I've": 'I have', "i'd've": 'i would have', "i'll've": 'i will have', "it'd": 'it would', "it'd've": 'it would have', "it'll've": 'it will have', "ma'am": 'madam', "mayn't": 'may not', "might've": 'might have', "mightn't've": 'might not have', "must've": 'must have', "mustn't've": 'must not have', "needn't": 'need not', "needn't've": 'need not have', "o'clock": 'of the clock', "oughtn't": 'ought not', "oughtn't've": 'ought not have', "sha'n't": 'shall not', "shan't've": 'shall not have', "she'd've": 'she would have', "she'll've": 'she will have', "should've": 'should have', "shouldn't've": 'should not have', "so've": 'so have', "so's": 'so as', "this's": 'this is', "that'd": 'that would', "that'd've": 'that would have', "there'd": 'there would', "there'd've": 'there would have', "here's": 'here is', "they'd've": 'they would have', "they'll've": 'they will have', "to've": 'to have', "we'd've": 'we would have', "we'll've": 'we will have', "what'll've": 'what will have', "when's": 'when is', "when've": 'when have', "where'd": 'where did', "where've": 'where have', "who'll've": 'who will have', "why's": 'why is', "why've": 'why have', "will've": 'will have', "won't've": 'will not have', "would've": 'would have', "wouldn't've": 'would not have', "y'all": 'you all', "y'all'd": 'you all would', "y'all'd've": 'you all would have', "y'all're": 'you all are', "y'all've": 'you all have', "you'd've": 'you would have', "you'll've": 'you will have'}
  text_tokenised = text.split()
  for i in text_tokenised:
    if(i in substitutes):
      text_list.append(substitutes[i])
    else:
      text_list.append(i)
  return ' '.join(text_list)

#Removing punctuations
def remove_punctuations(text):
  punctuations = '''!()-[|]`{};:'"\,<>./?@#$=+%^&*_~'''
  new_list = ['' if x in punctuations else x for x in text]
  new_list_final = []
  for token in new_list:
    new_token=""
    for char in token:
      if(char not in punctuations):
        new_token+=char
    if(len(new_token)!=0):
      new_list_final.append(new_token)
  return new_list_final

#Removing stopwords
def remove_stopwords(text):
  stopword = stopwords.words('english')
  new_list = [x for x in text if x not in stopword]
  return new_list

# #Converting number to words
# def num_to_words(text):
#   new_text = ' '.join([num2words.num2words(i) if i.isdigit() else i for i in text.split()])
#   return new_text
  
# #Stemming
# def stemming(text):
#   stemmer = PorterStemmer()
#   new_list = [stemmer.stem(x) for x in word_tokenize(text)]
#   return ' '.join(new_list)

#Tokenization
def tokenization(text):
  return word_tokenize(text)

def lemmatization(text):
  wordnet_lemmatizer = WordNetLemmatizer()
  new_list = [wordnet_lemmatizer.lemmatize(x) for x in text]
  return new_list


In [ ]:
#Combining all preprocessing steps together for a text
def text_preprocess(text):
  text = to_lower_case(text)
  text = substitute_text(text)
  text = tokenization(text)
  text = remove_stopwords(text)
  text = remove_punctuations(text)
  text = [x.strip() for x in text if len(x.strip())]
  text = lemmatization(text)
  return text

In [ ]:
def extract_date(text):
  return text[:10]

In [ ]:
print(extract_date("2022-02-14 00:11:44+00:00"))

2022-02-14


In [ ]:
text_preprocess("CommonSenseGunLaws,GunControlNow,GunSafes,GunSafety,GunRegistry,MSDStrong,EnoughIsEnough")

['commonsensegunlaws',
 'guncontrolnow',
 'gunsafes',
 'gunsafety',
 'gunregistry',
 'msdstrong',
 'enoughisenough']

In [ ]:
# headers
print(final_pd.head())

               author id                 created_at geo country country_code  \
0           1952510090.0  2022-02-14 00:11:44+00:00                            
1             20453105.0  2022-02-14 00:04:56+00:00                            
2  1492660754998247424.0  2022-02-14 00:39:55+00:00                            
3            117812637.0  2022-02-14 00:20:05+00:00                            
4  1490044524604928000.0  2022-02-14 00:10:26+00:00                            

  place_full_name place_name place_type                     id lang  ...  \
0                                        1493015060943454208.0   en  ...   
1                                        1493013352938934272.0   en  ...   
2                                        1493022155239534336.0   en  ...   
3                                        1493017163485044736.0   en  ...   
4                                        1493014734903382016.0   en  ...   

  reply_count retweet_count               source  \
0         

In [ ]:
# Creating preprocessed data column
final_pd['Preprocessed_Data'] = final_pd["tweet"] + " "+ final_pd["hashtags"] + " "+ final_pd["place_full_name"] + " " + final_pd["context_text"] 

In [ ]:
final_pd['Preprocessed_Data'] = final_pd['Preprocessed_Data'].map(text_preprocess)

In [ ]:
final_pd['Date_Only'] = final_pd['created_at'].map(extract_date)

In [ ]:
print(len(final_pd["Preprocessed_Data"]))

224213


In [ ]:
print(final_pd.head())

               author id                 created_at geo country country_code  \
0           1952510090.0  2022-02-14 00:11:44+00:00                            
1             20453105.0  2022-02-14 00:04:56+00:00                            
2  1492660754998247424.0  2022-02-14 00:39:55+00:00                            
3            117812637.0  2022-02-14 00:20:05+00:00                            
4  1490044524604928000.0  2022-02-14 00:10:26+00:00                            

  place_full_name place_name place_type                     id lang  ...  \
0                                        1493015060943454208.0   en  ...   
1                                        1493013352938934272.0   en  ...   
2                                        1493022155239534336.0   en  ...   
3                                        1493017163485044736.0   en  ...   
4                                        1493014734903382016.0   en  ...   

                source                                        

In [ ]:
print(final_pd["tweet"].iloc[0])
print(final_pd["hashtags"].iloc[0])
print(final_pd["Date_Only"].iloc[0])

ð¥ð¥ð¥ð¥ð¥ð¥ð¥â¤µï¸â¤µï¸â¤µï¸
Instead of treating the symptoms or aftermath of gun violence, we must go after the means, supply chain, the guns! #CommonSenseGunLaws #GunControlNow #GunSafes #GunSafety #GunRegistry #MSDStrong #EnoughIsEnough https://t.co/LjkEup24Dk
CommonSenseGunLaws,GunControlNow,GunSafes,GunSafety,GunRegistry,MSDStrong,EnoughIsEnough
2022-02-14


In [ ]:
print(final_pd["Preprocessed_Data"].iloc[0])

['ð\x9f\x92¥ð\x9f\x92¥ð\x9f\x92¥ð\x9f\x92¥ð\x9f\x92¥ð\x9f\x92¥ð\x9f\x92¥â¤µï¸\x8fâ¤µï¸\x8fâ¤µï¸\x8f', 'instead', 'treating', 'symptom', 'aftermath', 'gun', 'violence', 'must', 'go', 'mean', 'supply', 'chain', 'gun', 'commonsensegunlaws', 'guncontrolnow', 'gunsafes', 'gunsafety', 'gunregistry', 'msdstrong', 'enoughisenough', 'http', 'tcoljkeup24dk', 'commonsensegunlaws', 'guncontrolnow', 'gunsafes', 'gunsafety', 'gunregistry', 'msdstrong', 'enoughisenough']


In [ ]:
file1 = open("/content/drive/MyDrive/IR/twitter_base_preprocessed.pkl", "wb")
pickle.dump(final_pd, file1)
file1.close()

In [ ]:
final_pd.to_csv('twitter_base_final_tweet_dataset.csv')